# Linux Sysadmin DeCal Grade Processor
Last updated: Spring 2021

Created by bencuan

## About
This tool use Gradescope grade exports and generates an interactive grade report. (TODO: Automatically email students with their grade)

There is a precommit hook that automatically removes all output from the notebook, so don't worry about accidentally exposing sensitive data. Just don't commit the csv's themselves.

## Install
To run Jupyter notebooks, you can [install it here](https://jupyter.org/install) or use [JupyterHub](https://datahub.berkeley.edu/). vscode also seems to have ipynb support now so that's pretty cool. If running locally, ensure you have pandas (`pip install pandas`).

## Usage
1. Export grade CSV's from Gradescope and place them in the `data/` folder.
1. In the code box below, assign `beginner_path` and `advanced_path` to point to the respective csv's.
2. If the lab names changed, you may also need to modify the `beginner_labs` and `advanced_labs` variables. Make them a list with each entry being a string containing one lab name (e.g. 'b1')
4. Run all of the cells!
5. 
5. The final cell exports the result to the file `data/grades_compiled.csv`.

## IMPORTANT NOTE
This will probably produce a LOT of NP's since many students have submitted late lab requests, etc. Go through each NP manually to ensure that individual cases have been dealt with.

In [ ]:
# IMPORTS
import pandas as pd

In [ ]:
# CONFIGURATION
pd.option_context('display.max_rows', None, 'display.max_columns', None) # display full chart

beginner_path = 'Beginner_Spring_2021_grades.csv'
advanced_path = 'Advanced_Spring_2021_grades.csv'

beginner_labs = [f'b{i}' for i in range(1, 11)] + ['11']
advanced_labs = [f'a{i}' for i in range(1, 11)] + ['11']

COMPLETE_LAB_MIN = 9 # At least this number of labs turned in overall for P
LATE_LAB_MAX = 5 # At least this number of labs turned in for P



In [ ]:
# Read grade data
b_grades = pd.read_csv(f'data/{beginner_path}')
a_grades = pd.read_csv(f'data/{advanced_path}')

# Make human readable

def process(grades, labs):
    statuses = {0: 'INC', 1: 'LATE', 2: 'DONE'}
    cleaned = grades.loc[:,['Name', 'SID', 'Email']]
    cleaned.insert(3, 'Completed', 0)
    cleaned.insert(4, 'Num Late', 0)
    cleaned.insert(3, 'Grade', 'NP')

    for lab in labs:
        cleaned[lab] = grades[f'Lab {lab}'].ge(grades[f'Lab {lab} - Max Points'] / 2).astype(int)
        cleaned.loc[cleaned[lab] == 1, lab] = cleaned[lab].astype(int) + (grades[f'Lab {lab} - Lateness (H:M:S)'] == '00:00:00').astype(int)
        cleaned[lab] = cleaned[lab].map(lambda x: statuses[x])
        cleaned['Completed'] += cleaned[lab] == 'DONE'
        cleaned['Completed'] += cleaned[lab] == 'LATE'
        cleaned['Num Late'] += cleaned[lab] == 'LATE'
        cleaned.loc[(cleaned['Completed'] >= COMPLETE_LAB_MIN) & (cleaned['Num Late'] <= LATE_LAB_MAX), 'Grade'] = 'P'

    return cleaned

b_cleaned = process(b_grades, beginner_labs)
a_cleaned = process(a_grades, advanced_labs)

In [ ]:
# Show uncleaned data
a_grades.head(5)

In [ ]:
b_grades.head(5)

In [ ]:
# Show beginner grades
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(b_cleaned)

In [ ]:
# Show advanced grades
display(a_cleaned)

In [ ]:
# Show not-a-number warnings (should be manually addressed)
b_cleaned = b_cleaned.fillna('ERROR')
a_cleaned = a_cleaned.fillna('ERROR')
a_cleaned[~a_cleaned['SID'].astype(str).str.isnumeric()]

In [ ]:
b_cleaned[~b_cleaned['SID'].astype(str).str.isnumeric()]

In [ ]:
# Drop the bad rows
a_cleaned = a_cleaned[a_cleaned['SID'].astype(str).str.isnumeric()]
b_cleaned = b_cleaned[b_cleaned['SID'].astype(str).str.isnumeric()]

## Export Grades for Submission
The script below takes in an empty grade sheet (contact Hilfinger or Cindy to get) and fills it in with P's and NP's. It is recommended that you manually look over the output and enter comments for all of the NP's.

All of the above cells must be run before this section.

In [ ]:
EMPTY_ROSTER_PATH = 's2021-grades-blank.csv'
blank = pd.read_csv(f'data/{EMPTY_ROSTER_PATH}')
blank = blank.drop(columns = ['Grade'])
blank

In [ ]:
a_brief = a_cleaned.loc[:, ['SID', 'Grade']]
b_brief = b_cleaned.loc[:, ['SID', 'Grade']]
b_brief['SID'] = pd.to_numeric(b_brief.SID)
a_brief['SID'] = pd.to_numeric(a_brief.SID)
combined = a_brief.merge(b_brief, how='outer', on='SID')
filled = blank[['SID', 'Grading Basis', 'Name']]
filled = filled.merge(combined, how='left', on='SID')
filled['Grade_x'] = filled['Grade_x'].fillna(filled['Grade_y'])
filled = filled.rename(columns = {'Grade_x': 'Grade'})
filled = filled.drop(columns = ['Grade_y'])
filled = filled[['SID', 'Name', 'Grade', 'Grading Basis']]
filled['Comments'] = ''

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(filled)

In [ ]:
# Export
filled.to_csv('data/grades_compiled.csv')